In [19]:
import pandas as pd 
from dotenv import load_dotenv
from langchain.document_loaders import DirectoryLoader,TextLoader
import os 
import google.generativeai as genai
from langchain.prompts import PromptTemplate
from langchain_google_genai import GoogleGenerativeAI
import docx
import markdown
import PyPDF2
import helpers
import mlflow
from IPython.core.display import HTML

In [20]:
RESUME_PATH = 'Insert Path here'
resume = helpers.extract_text_from_pdf(RESUME_PATH)
resume

'Shehab Diab\n+20 1100121212 - shehabgehaddiab@gmail.com - Cairo, Egypt\nLinkedIn Github Datacamp Kaggle Leetcode\nEducation\nBSc Computer Science, AI Major Cairo University Cairo, Egypt 2021-2025\nRelevant Courses: Software Engineering, Database, Web Development, Machine Learning, Supervised Learning, Signals &\nSystems, Discrete Math, Linear Algebra & Calculus, Statistics, Algorithms & Data Structures, OOP\nProjects\nFlight Reservation System Application - GitHub University Assignment\n•Users have accounts and can Search/Book/Manage flights, and Admins can Create/Modify/Cancel flights and aircrafts.\n•Using : C#, WinForms, and MS SQL server, Regex, Stored Procedures.\nEvolutionary Computation Algorithms - GitHub University Assignment\n•Solved Travelling Salesman Problem using Ant Colony Optimization, trained a Genetic algorithm to play Tetris on its own.\n•Using : NumPy, Pandas, Matplotlib, Seaborn, Pygame, Manim.\nRice Type Detection Project - Kaggle Personal Project\n•Classifying 5

In [29]:
# here i create the prompt that will be feed to the llm as input.
PROMPT_TEMPLATE = '''
        You are a resume editor AI tool, 
        your role is to modify the given and based on the given job descripiton 
        Notes: 
        - You can reorder the projects and the experince based on the wanted job description.
        - The technical skills and soft skill are unchanged in the modified resume.
        - Make sure that you handle the job title in the resume to be as the job description job title.
        - If in the resume the GPA was above 3 mention it, otherwise do not mention the GPA and remove it.
        - You can change words of the resume to be more convienent.
        - Do not add non existent skills or experiences to the main resume
        - Do not create new titles, skills, experiences in the modified resume
        - remove any entity that is not related to the job description
        here is the given resume: 
        {resume}
        -----------------------------
        and here is the job description: 
        {job_description}
        
        Based on the above job description try to edit and arrange the resume, to fit the given job description.
    '''

prompt = PromptTemplate(
    template = PROMPT_TEMPLATE,
    input_variables = ['resume','job_description'] # the input will be formated to the prompt like placeholders
)

In [22]:
# creating the LLM model. 
load_dotenv()
API_KEY=os.environ['GOOGLE_API_KEY']

if API_KEY == None:
    API_KEY = 'Give User Error' ## secret do not share it or use it too much 

genai.configure(api_key=API_KEY)
TEMPRATURE = 0.5
llm = GoogleGenerativeAI(google_api_key=API_KEY,model='gemini-1.5-flash', temprature=TEMPRATURE) ## here the temprature is how much you want your model to be creative and choose random choices.

In [30]:
job_description = '''
    This is an on-site part time role for a Full Stack/Python Instructor at AMIT Learning. The Instructor will be responsible for planning and delivering high-quality training sessions on Full Stack and Python development. Daily tasks include developing course materials, conducting hands-on training sessions, and providing mentorship and guidance to students.

Qualifications
 
⦁ University degree in Computer Science, Software Engineering, or similar.
⦁ 2 to 4 years of experience in back-end development using Python.
⦁ Proven experience in building server-side applications and RESTful APIs.
⦁ Proficient in Python frameworks like Django or Flask.
⦁ Familiarity with front-end technologies like HTML, CSS, and JavaScript.
⦁ Strong understanding of database management (SQL, NoSQL) and ORM tools.
⦁ Experience with cloud platforms like AWS, Azure, or Google Cloud.
⦁ Working experience with version control systems (e.g., Git).
⦁ Knowledge of Agile methods and practices.
⦁ Very good English.
⦁ Presentable with excellent communication skills.


Responsibilities:


Conducting and delivering direct training sessions for internal and external clients.
Prepare and evaluate diploma materials, presentations, assignments, graduation projects and the final exam.
Answer students’ questions and provide quick support.
'''
result = llm.invoke(prompt.format(resume=resume,job_description=job_description))

In [31]:
new_resume = HTML(markdown.markdown(result))
new_resume

In [25]:
html_resume_path = helpers.write_html_file(new_resume)

In [26]:
import dagshub
dagshub.init(repo_owner='shehabgehaddiab', repo_name='Resume_Editor', mlflow=True)

Initialized MLflow to track repo "shehabgehaddiab/Resume_Editor"

Repository shehabgehaddiab/Resume_Editor initialized!

In [27]:
mlflow.set_tracking_uri("https://dagshub.com/shehabgehaddiab/Resume_Editor.mlflow")
mlflow.set_experiment("CV Generation")

<Experiment: artifact_location='mlflow-artifacts:/228bd857794a4168b1f51dd3312e5838', creation_time=1727439173052, experiment_id='0', last_update_time=1727439173052, lifecycle_stage='active', name='CV Generation', tags={}>

In [32]:
with mlflow.start_run():
    mlflow.log_param("prompt", PROMPT_TEMPLATE)
    mlflow.log_artifact(RESUME_PATH, artifact_path="resumes/old/shehab")
    mlflow.log_artifact(html_resume_path, artifact_path="resumes/new/shehab")
    mlflow.log_param("job_description", job_description)
    mlflow.log_param("tempreture", TEMPRATURE)

    mlflow.log_param("CV Generator Version", "V1")


2024/09/27 15:23:25 INFO mlflow.tracking._tracking_service.client: 🏃 View run victorious-gull-621 at: https://dagshub.com/shehabgehaddiab/Resume_Editor.mlflow/#/experiments/0/runs/f8ebd80cfc284627bfdd8a241be2a1ed.
2024/09/27 15:23:25 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: https://dagshub.com/shehabgehaddiab/Resume_Editor.mlflow/#/experiments/0.
